In [1]:
import pandas as pd
import numpy as np

In [128]:
pos=pd.read_csv('new_weibo_data_clean.csv',names=['idx','content'])
neg=pd.read_csv('new_weibo_data_clean0.csv',names=['idx','content'])

In [130]:
# count=0
# # train_x=train_x[~np.isnan(train_x).any(axis=1)]
# for line in pos.values:
#     count+=1
#     if np.isnan(line):
#         print(count)

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [20]:
from gensim.models.word2vec import Word2Vec
model = Word2Vec.load('wiki_w2v.model')

In [40]:
# output_file_name = 'weibo1.txt'
# output_file = open(output_file_name, 'w', encoding='utf-8')
# label=1
# for line in pos.values:
#     length=0
#     vecs=np.zeros((1,300))
#     newline=line[0].split(' ')
#     for i in newline:
#         try:
#             vecs+=model[i].reshape((1,300))
#             length+=1
#         except KeyError:
#             continue  

#     otp = "{0} {1}\n".format((vecs/length),label)
#     output_file.write(otp)

f:\python\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
f:\python\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


In [132]:
pos_arr=[]
pos_la=[]
for line in pos.values:
    length=0
    vecs=np.zeros((1,300))
    newline=line[0].split(' ')
    for i in newline:
        try:
            vecs+=model[i].reshape((1,300))
            length+=1
        except KeyError:
            continue  
    pos_arr.append(vecs/length)
    pos_la.append(float(1))
pos_arr=np.array(pos_arr).reshape(-1,300)
pos_la=np.array(pos_la)

f:\python\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
f:\python\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


In [133]:
neg_arr=[]
neg_la=[]
for line in neg.values:
    length=0
    vecs=np.zeros((1,300))
    newline=line[0].split(' ')
    for i in newline:
        try:
            vecs+=model[i].reshape((1,300))
            length+=1
        except KeyError:
            continue  
    neg_arr.append(vecs/length)
    neg_la.append(float(0))
neg_arr=np.array(neg_arr).reshape(-1,300)
neg_la=np.array(neg_la)

f:\python\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
f:\python\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


In [137]:
neg_la=neg_la.reshape((-1,1))
pos_la=pos_la.reshape((-1,1))

In [139]:
#合并与空值去除
data_pos=np.concatenate((pos_arr,pos_la),axis=1)
data_neg=np.concatenate((neg_arr,neg_la),axis=1)
data_pos=data_pos[~np.isnan(data_pos).any(axis=1)]
data_neg=data_neg[~np.isnan(data_neg).any(axis=1)]

In [142]:
dataX=np.concatenate((data_pos[:,:-1],data_neg[:,:-1]),axis=0)
dataY=np.concatenate((data_pos[:,-1],data_neg[:,-1]),axis=0)

In [143]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(dataX,dataY,test_size=0.1)


In [144]:
print(train_x.shape)
print(test_x.shape)
print(train_y.shape)
print(test_y.shape)

(107676, 300)
(11964, 300)
(107676,)
(11964,)


In [145]:
from sklearn import neural_network
clf=neural_network.MLPClassifier()
clf.fit(train_x,train_y)
clf.score(test_x,test_y)

f:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.8552323637579405

In [146]:
from sklearn.decomposition import PCA
pca = PCA(n_components=64)  
pca_fitted=pca.fit(train_x)
train_x=pca_fitted.transform(train_x)
test_x=pca_fitted.transform(test_x)

In [ ]:
import pickle
from sklearn.svm import SVC
#设计不同参数的分类器并保存模型
models=[]
for k in ['linear','poly','rbf','sigmoid']:
    for c in [0.1,1,2,8]:
        clf = SVC(kernel=k,C=c)
        clf.fit(train_x,train_y)
        model_name=k+'_'+str(c)+'_'+'svm'+'.pickle'
        models.append(model_name)
        with open(model_name, 'wb') as fw:
            pickle.dump(clf, fw)



In [ ]:
for m in models:
    with open(m, 'rb') as fr:
        new_svm = pickle.load(fr)
        print (m+str(new_svm.score(test_x,test_y)))


In [ ]:
#进步进行降维都可以
import pickle
from sklearn.linear_model import LogisticRegression
lr_models=[]
for s in ['liblinear','lbfgs','newton-cg','sag']:
    for m in [1000,3000,5000]:    #这三种迭代次数的结果是一样的
        lr=LogisticRegression(max_iter=m,solver=s) 
        lr.fit(train_x,train_y)
        lr_model_name=s+'_'+str(m)+'_'+'logistic'+'.pickle'
        lr_models.append(lr_model_name)
        with open(lr_model_name, 'wb') as fw:
            pickle.dump(lr, fw)


In [ ]:
for m in lr_models:
    with open(m, 'rb') as fr:
        new_lr = pickle.load(fr)
        print (m+str(new_lr.score(test_x,test_y)))

#liblinear_1000_logistic.pickle0.8615847542627884  #与svc相比速度快很多

In [ ]:
# liblinear_1000_logistic.pickle0.8615847542627884
# liblinear_3000_logistic.pickle0.8615847542627884
# liblinear_5000_logistic.pickle0.8615847542627884
# lbfgs_1000_logistic.pickle0.8615011701771983
# lbfgs_3000_logistic.pickle0.8615011701771983
# lbfgs_5000_logistic.pickle0.8615011701771983
# newton-cg_1000_logistic.pickle0.8615011701771983
# newton-cg_3000_logistic.pickle0.8615011701771983
# newton-cg_5000_logistic.pickle0.8615011701771983
# sag_1000_logistic.pickle0.8615011701771983
# sag_3000_logistic.pickle0.8615011701771983
# sag_5000_logistic.pickle0.8615011701771983